Comprehensive Analysis of the Indian Premier League and Projecting the Optimal Squad for the 2025 Season

In [1]:
# Load Dataset
import pandas as pd
dataset = pd.read_csv("Dataset/IPL_Meta_Data.csv")
df = dataset
metadata = dataset
# dataset.head()

C:\Users\niran\AppData\Local\Temp\ipykernel_19968\3160474569.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("Dataset/IPL_Meta_Data.csv")


This dataset contains information about the IPL data, contains ball by ball data from 2008 to 2024. 


In [2]:
# This is the sample data
dataset.head(1)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,1426312,2024,5/26/2024,"MA Chidambaram Stadium, Chepauk, Chennai",1,0.1,Sunrisers Hyderabad,Kolkata Knight Riders,Abhishek Sharma,TM Head,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We are cleaning the data and trying to create different tables namely:
Player Table 
Batsman table,
Bowlers table, 
Fielding table, 
matches table

In [3]:
# Column names and Data types 
dataset.dtypes

match_id                    int64
season                     object
start_date                 object
venue                      object
innings                     int64
ball                      float64
batting_team               object
bowling_team               object
striker                    object
non_striker                object
bowler                     object
runs_off_bat                int64
extras                      int64
wides                     float64
noballs                   float64
byes                      float64
legbyes                   float64
penalty                   float64
wicket_type                object
player_dismissed           object
other_wicket_type         float64
other_player_dismissed    float64
dtype: object

1. Players Table: 
table name: player_info

In [86]:
unique_players = pd.concat([df['striker'], df['non_striker'], df['bowler']]).drop_duplicates().sort_values()

player_info = pd.DataFrame(columns=['player', 'seasons'])

player_list = []
seasons_list = []

for player in unique_players:
    player_rows = df[(df['striker'] == player) | (df['non_striker'] == player) | (df['bowler'] == player)]
    seasons = ', '.join(sorted(set(player_rows['season'].astype(str))))
    player_list.append(player)
    seasons_list.append(seasons)

player_info['player'] = player_list
player_info['seasons'] = seasons_list

player_info['seasons'] = player_info['seasons'].apply(lambda x: ', '.join(sorted(set(x.split(', ')), key=x.split(', ').index)))

In [89]:
# Count the number of seasons a player played
player_info['num_seasons'] = player_info['seasons'].apply(lambda x: len(x.split(', ')))


In [91]:
player_info

Runs scored by a player in each innings ball by ball is stored in : player_runs

In [99]:
# Calculate the ball by ball runs scored by each player in each innings
player_runs = df.groupby(['match_id', 'striker'])['runs_off_bat'].apply(list).reset_index()
player_runs['runs_off_bat'] = player_runs['runs_off_bat'].apply(lambda x: ' '.join(map(str, x)))
player_runs.columns = ['match_id', 'player', 'ball_by_ball_runs']

In [100]:
player_runs

,match_id,player,ball_by_ball_runs
0,335982,AA Noffke,0 0 0 1 1 4 1 0 0 0 1 1
1,335982,B Akhil,0 0
2,335982,BB McCullum,0 0 0 0 0 0 0 4 4 6 4 0 4 1 0 6 0 6 4 0 1 1 1 ...
3,335982,CL White,0 1 0 0 1 1 1 0 2 0
4,335982,DJ Hussey,0 1 1 4 1 1 0 1 0 2 1 0
...,...,...,...
16510,1426312,SP Narine,6 0
16511,1426312,SS Iyer,4 1 1
16512,1426312,Shahbaz Ahmed,2 0 0 6 0 0 0
16513,1426312,TM Head,0


Total runs scored and runs scored in each innings is stored in: player_runs_sum

In [131]:
# Sum the runs scored by each player and calculate the total runs scored

player_runs_sum = df.groupby(['match_id', 'striker'])['runs_off_bat'].apply(list).reset_index()
player_runs_sum['runs_off_bat'] = player_runs_sum['runs_off_bat'].apply(lambda x: sum(x))
player_runs_sum.columns = ['match_id', 'player', 'runs_scored_each_inning']

# Create a new DataFrame with the total runs scored by each player
total_runs_sum = player_runs_sum.groupby('player')['runs_scored_each_inning'].sum().reset_index()
total_runs_sum.columns = ['player', 'total_runs_scored']

# Merge the two DataFrames
player_runs_sum = pd.merge(player_runs_sum, total_runs_sum, on='player', how='left')

In [132]:
# Calculate the balls faced in each inning
player_runs_sum['balls_faced_each_inning'] = player_runs['ball_by_ball_runs'].apply(lambda x: len(str(x).split()))

In [135]:
# Calculate the dotballs in each inning
dot_balls = df[df['runs_off_bat'] == 0].groupby(['match_id', 'striker'])['ball'].count().reset_index()
dot_balls.columns = ['match_id', 'player', 'dot_balls']
dot_balls
# Merged dotball column into players runs sum table
# player_runs_sum = pd.merge(player_runs_sum, dot_balls[['player', 'dot_balls']], on='player', how='left')


,match_id,player,dot_balls
0,335982,AA Noffke,6
1,335982,B Akhil,2
2,335982,BB McCullum,23
3,335982,CL White,5
4,335982,DJ Hussey,4
...,...,...,...
15727,1426312,Rahmanullah Gurbaz,21
15728,1426312,SP Narine,1
15729,1426312,Shahbaz Ahmed,5
15730,1426312,TM Head,1


In [133]:
player_runs_sum

,match_id,player,runs_scored_each_inning,total_runs_scored,balls_faced_each_inning
0,335982,AA Noffke,9,9,12
1,335982,B Akhil,0,76,2
2,335982,BB McCullum,158,2882,77
3,335982,CL White,6,971,10
4,335982,DJ Hussey,12,1322,12
...,...,...,...,...,...
16510,1426312,SP Narine,6,1534,2
16511,1426312,SS Iyer,6,3131,3
16512,1426312,Shahbaz Ahmed,8,536,7
16513,1426312,TM Head,0,772,1
